In [1]:
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
book_path = '../data/book_score.csv'
movie_path = '../data/movie_score.csv'
book_df = pd.read_csv(book_path, encoding='utf-8')
movie_df = pd.read_csv(movie_path, encoding='utf-8')
book_df.head()
print(book_df.nunique())
print(movie_df.nunique())

User      4419
Book      1200
Rate         6
Time    634776
Tag      59960
dtype: int64
User       1023
Movie      1200
Rate          6
Time     712198
Tag      110708
dtype: int64


In [3]:
# print(movie_df[movie_df["Rate"] == 0].nunique())
print(book_df["Rate"].value_counts())

Rate
0    233447
4    165052
5    131574
3     92447
2     11296
1      3438
Name: count, dtype: int64


In [4]:
book_df.rename(columns={'Book': 'Item'}, inplace=True)
movie_df.rename(columns={'Movie': 'Item'}, inplace=True)

book_df.drop(book_df[book_df["Rate"] == 0].index, inplace=True)
movie_df.drop(movie_df[movie_df["Rate"] == 0].index, inplace=True)

print(movie_df["Rate"].value_counts())

Rate
4    203661
3    162926
5    115492
2     32510
1      9059
Name: count, dtype: int64


In [5]:
from sklearn.model_selection import train_test_split

data = movie_df
train_data, test_data = train_test_split(data, test_size=0.5, random_state=42)

In [6]:
matrix = train_data.pivot_table(index='User', columns='Item', values='Rate')
user_mean = matrix.mean(axis=1)
matrix = matrix.subtract(matrix.mean(axis=1), axis="rows")
user_mean.head()

User
1000068    3.294118
1000174    3.246667
1000263    2.000000
1000905    4.160000
1001093    3.706667
dtype: float64

In [7]:
user_similarity = matrix.T.corr(method="pearson")
user_similarity.head()

User,1000068,1000174,1000263,1000905,1001093,1002057,1002987,1003080,1004613,1005928,...,36855984,37829070,39742649,42444500,43828497,44605628,45987769,46788722,48190738,48360271
User,,,,,,,,,,,,,,,,,,,,,
1000068,1.000000,1.000000,NaN,0.828079,0.315264,0.709299,0.369274,0.207514,0.763763,0.880141,...,-5.934392e-17,NaN,NaN,0.615385,NaN,NaN,0.707107,NaN,NaN,NaN
1000174,1.000000,1.000000,NaN,0.257039,0.227085,0.262536,0.057639,0.386178,0.345861,0.213338,...,4.349962e-01,0.557926,-0.062017,0.524704,0.871421,0.611590,0.264965,NaN,NaN,NaN
1000263,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1000905,0.828079,0.257039,NaN,1.000000,0.347644,0.132445,0.312092,0.374112,0.188312,0.251513,...,3.573042e-01,-0.412780,-0.177555,0.435665,0.000000,0.259310,0.196282,NaN,NaN,NaN
1001093,0.315264,0.227085,NaN,0.347644,1.000000,0.340371,0.411297,0.257495,0.498377,0.226428,...,4.000265e-01,0.165152,-0.210042,0.416306,0.495222,0.377014,0.366416,NaN,-1.0,NaN


In [8]:
# use the similarity matrix to predict the user's rating for an item
similar_users_storage = dict()


def predict(user, item, similarity_threshold=0.5, similar_users_num=30):
    if (user not in matrix.index):
        return user_mean.mean()

    user_rating = matrix.loc[user, item]
    if np.isnan(user_rating):

        item_rating = matrix[item]
        item_rating = item_rating.dropna()

        if (similar_users_storage.get(user) is None):
            similar_users = user_similarity[user].dropna()
            similar_users = similar_users[similar_users > similarity_threshold]
            similar_users = similar_users.sort_values(ascending=False)
            similar_users_storage[user] = similar_users
        else:
            similar_users = similar_users_storage[user]

        # Only consider similar users who have rated the item
        common_users = list(
            set(similar_users.index).intersection(set(item_rating.index)))
        common_users.sort(key=lambda x: similar_users[x], reverse=True)

        common_users = common_users[:similar_users_num if len(common_users) >
                                    similar_users_num else len(common_users)]
        item_rating = item_rating.loc[common_users]

        if len(item_rating) == 0:
            return user_mean[user]
        else:
            return item_rating.mean() + user_mean[user]
    else:
        return user_rating

In [9]:
from sklearn.metrics import ndcg_score


def compute_ndcg(group):
    true_ratings = group['true_rating'].tolist()
    pred_ratings = group['predict_rating'].tolist()
    return ndcg_score([true_ratings], [pred_ratings], k=50)

In [10]:
from tqdm.notebook import tqdm

predict_rating = []
for idx, (user, item, true_rating, time, tag) in tqdm(test_data.iterrows()):
    # print(user, item, true_rating, time, tag)
    predict_rating.append(
        predict(user, item, similarity_threshold=0.4, similar_users_num=50))
    # print(true_rating, predict_rating)

results_df = pd.DataFrame({
    'user': test_data['User'],
    'true_rating': test_data['Rate'],
    'predict_rating': predict_rating
})

print(results_df.head())


0it [00:00, ?it/s]

           user  true_rating  predict_rating
476771  1391511            4        4.124862
214632  1052753            4        4.525523
581696  1435341            3        2.454506
246040  1935228            5        3.893728
194159  1494167            4        4.634479


In [11]:
# 助教是这么写的，就很神奇, 但是这样对只有一个项目求ndcg会报错
# ndcg_scores = results_df.groupby('user').apply(compute_ndcg)
ndcg_scores = results_df[results_df.groupby('user')['user'].transform('count')
                         > 1].groupby('user').apply(compute_ndcg)

avg_ndcg = ndcg_scores.mean()
print(f"Average NDCG score: {avg_ndcg}")

Average NDCG score: 0.9180497961365143
